In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import numpy as np
from scipy.optimize import fsolve, minimize
from scipy.integrate import solve_ivp
from matplotlib import animation
from IPython.display import HTML
import matplotlib
from matplotlib import patches
import matplotlib.pyplot as plt
%matplotlib inline
import time
matplotlib.rcParams['animation.embed_limit'] = 2**128

A homogenious ellipse of mass *m* and parameters *a, b* rolls on an uneven street without slipping of jumping. A particle of mass $m_o$ may be attached anywhere within th ellipse.\
The street is a 'line' in the X/Y plane, gravitation points in the negative Y - direction.


Note: the special case of the ellipse running on a horizontal line is solved explicitly here:
https://www.mapleprimes.com/DocumentFiles/210428_post/rolling-ellipse.pdf

**Parameters**
- *N*: inertial frame
- *A*: frame fixed to the ellipse
- $P_0$: point fixed in *N*
- *Dmc*: center of the ellipse
- *CP*: contact point
- $P_o$: location of the particle fixed to the ellipse


- *q, u*: angle of rotation of the ellipse, its speed
- $x, u_x$: X coordinate of the contact point CP, its speed
- $m_x, m_y, um_x, um_y$: coordinates of the center of the ellipse, its speeds


- $m, m_o$: mass of the ellipse, of the particle attached to the ellipse
- *a, b*: semi axes of the ellipse
- *amplitude, frequenz*: parameters for the street.
- $i_{ZZ}$: moment of inertia of the ellipse around the Z axis
- $\alpha, \beta$: determine the location of the particel w.r.t. Dmc
- $CP_x, CP_y$: needed only to set up the equation for the ellipse

In [ ]:
start = time.time()

m, mo, g, CPx, CPy, a, b, iZZ, alpha, beta = sm.symbols('m, mo, g, CPx, CPy, a, b, iZZ, alpha, beta')
mx, my, umx, umy = me.dynamicsymbols('mx, my, umx, umy')
q, x, u, ux = me.dynamicsymbols('q, x, u, ux')

t = sm.symbols('t')
amplitude, frequenz = sm.symbols('amplitude, frequenz')

N, A = sm.symbols('N, A', cls=me.ReferenceFrame)
P0, Dmc, CP, Po = sm.symbols('P0, Dmc, CP, Po', cls=me.Point)

P0.set_vel(N, 0.)

A.orient_axis(N, q, N.z)
A.set_ang_vel(N, u*N.z)

Model the street.\
It is a parabola, open to the top, with superimposed sinus waves.\
Then I calculate the formula for its osculating circle, the formula of which I found in the internet.

In [ ]:
#Modeling the street
#============================================
rumpel = 3  # the higher the number the more 'uneven the street'
#============================================

strasse = sum([amplitude/j * sm.sin(j*frequenz * CPx) for j in range(1, rumpel)])
strassen_form = (frequenz/2. * CPx)**2
gesamt = strassen_form + strasse
gesamtdx = gesamt.diff(CPx)

r_max = ((sm.S(1.) + (gesamt.diff(CPx))**2 )**sm.S(3/2)/gesamt.diff(CPx, 2)).subs({CPx: x})

**Find the center of the ellipse**\
To do this, I proceed as follows:
- I set up the general equation for a ellipse which is rotated about an angle $\phi$ and its center is at an arbitrary point (mx, my). With this equation, given a point on the circumference of the ellipse and its rotation $\phi$ one can numerically calculate mx, my.
- I pick the point of the circumference of the ellipse in such a way, that it is the *contact point* with the street. A *necessary* condition for this is, that the tangent of the ellipse at this point $x_0$ is parallel to the tangent of the street at the same point.
- in order to get $\frac{d}{dx}ellipse(y, a, b, $\phi$)$ I first solve the equation of the ellipse for *y*, that is *y = function(x, a, b, $\phi$)*, and then I calculate $\frac{d}{dx}function(..)$.

Soving the equation of the ellipse for *y* has two solutions. I picked the right one by trial and error.
    

In [ ]:
# This is to get the center of the ellipse, given the contact point and its derivative
#=============================================
Diag = sm.diag(1./a**2, 1./b**2, 1.)
#A_matrix = sm.Matrix([[sm.cos(q1), sm.sin(q1)], [-sm.sin(q1), sm.cos(q1)]]) # rotational matrix
A1 = A.dcm(N)
print('A1: ', A1, '\n')
vektor =sm.Matrix([(CPx - mx), (CPy - my), 0.])
ellipse = ((vektor.T * A1.T) * Diag * (A1 * vektor)) - sm.Matrix([sm.S(1.)])
ellipse[0, 0] = ellipse[0, 0].simplify()
print('ellipse free symbols', ellipse.free_symbols)
print('ellipse DS', me.find_dynamicsymbols(ellipse), '\n')

ellipse_x = sm.solve(ellipse, CPy) # ellipse solved for y = f(x), so d/dt(f(x)) may be calculated
ellipse_dx = (ellipse_x[0][0].diff(CPx)).simplify()

print('ellipse_dx free symbols', ellipse_dx.free_symbols)
print('ellipse_dx DS', me.find_dynamicsymbols(ellipse_dx), '\n')

# Richtung1 gives the center of the ellipsoid, given the contact points and its rotation q1
richtung = (sm.Matrix([ellipse[0, 0], ellipse_dx - gesamtdx]).subs({CPy: gesamt})).subs({CPx: x})
print('Richtung free symbols', richtung.free_symbols)
print('Richtung DS', me.find_dynamicsymbols(richtung), '\n')

**Speed constraints**\
Above I calculated the (non linear) equation for the coordinates *mx, my* of the center of the ellipse. From this *configuration constraint* I calculate the resulting *speed constraints* the usual way:
- umx = loesung[0]
- umy = loesung[1]

*linsolve* does not work here practically, it takes too long.
loesung also contains $\frac{d}{dt}x(t)$, and I substitute as soon as $rhs_x$ is available, see below.

In [ ]:
CP.set_pos(P0, x*N.x + gesamt*N.y)
Dmc.set_pos(P0, mx*N.x + my*N.y)

richtung_dict = {sm.Derivative(mx, t): umx, sm.Derivative(my, t): umy, sm.Derivative(q, t): u}

richtungdt = richtung.diff(t).subs(richtung_dict)

matrix_A = richtungdt.jacobian((umx, umy))
vector_b = richtungdt.subs({umx: 0., umy: 0.})
loesung = matrix_A.LUsolve(-vector_b)
print('loesung DS', me.find_dynamicsymbols(loesung))

**Relationship of x(t) to q(t)**:



Obviously, $ x(t) = function(q(t), gesamt(x(t)), a, b) $.
When the ellipse is rotated from 0 to $q$, the arc length is $\int_{0}^{q(t)} \sqrt{a^2sin(k)^2 + b^2cos(k)^2}\,dk \ $.

The arc length of a function f(k(t)) from 0 to $x(t)$ is: $ \int_{0}^{x(t)} \sqrt{1 +  (\frac{d}{dx}(f(k(t))^2} \,dk \ $ (I found all this in the internet)

This gives the sought after relationship between $q(t)$ and $x(t)$:

$\int_{0}^{q(t)} \sqrt{a^2sin(k)^2 + b^2cos(k)^2}\,dk \  \   =  \int_{0}^{x(t)} \sqrt{1 + (\frac{d}{dk}(gesamt(k(t))^2}\,dk \ $, differentiated w.r.t *t*:
- $\sqrt{a^2sin(q(t))^2 + b^2cos(q(t))^2}\  \cdot (-\frac{d}{dt}q(t))  = \sqrt{1 + (\frac{d}{dx}(gesamt(x(t))^2} \cdot d/dt(x(t)) $, that is solved for $\frac{d}{dt}(x(t))$:


- $\frac{d}{dt}(x(t)) = \dfrac{-\sqrt{a^2sin(q(t))^2 + b^2cos(q(t))^2}} {\sqrt{1 + (\frac{d}{dx}(gesamt(x(t))^2}} \cdot u(t)$

The - sign is a consequence of the 'right hand rule' for frames. This is the sought after first order differential equation for $x(t)$.

I calculate the **speed of Dmc** using *Dmc.pos_from(P0).diff(t, N)* This contains terms of $\frac{d}{dt}x(t)$.
For some reason I do not understand, they **must** be replaced by $rhs_x$ right here! If I do not substitute $\frac{d}{dt}x(t)$ right here, this term gets into the force vector function, and cannot be removed there anymore - at least I found no way of doing it.\
Actually, if I do not replace $\frac{d}{dt}x(t)$ right here and I try to get the *dynamic symbols* of the force vector, they do **not** contain $\frac{d}{dt}x(t)$ - but lambdifying with cse = True will throw an error.  

In [ ]:
sigma = sm.sqrt((a*sm.sin(q))**2 + (b*sm.cos(q))**2)
subs_dict1 = {sm.Derivative(q, t): u, CPx: x}
rhsx = (-u * sigma/sm.sqrt(1. + gesamtdx**2)).subs(subs_dict1)
print('rhsx DS', me.find_dynamicsymbols(rhsx))
print('rhsx free symbols', rhsx.free_symbols)
print(rhsx.count_ops(visual=False))

loesung = loesung.subs({sm.Derivative(x, t): rhsx})
print('loesung DS', me.find_dynamicsymbols(loesung))
Dmc_dict = {umx: loesung[0], umy: loesung[1], sm.Derivative(x, t): rhsx}
Dmc.set_vel(N, (Dmc.pos_from(P0).diff(t, N)).subs(richtung_dict).subs(Dmc_dict))

Po.set_pos(Dmc, a*alpha*A.x + b*beta*A.y)
Po.v2pt_theory(Dmc, N, A)

print('v(Dmc) DS:', me.find_dynamicsymbols(Dmc.vel(N), reference_frame=N))

**Kane's equations**\
There is nothing special here.\
I need to solve the differential equation for $x(t)$ numerically, so this is added to the force vector. Of course the mass matrix must the enlarged correspondingly.

In [ ]:
start1 = time.time()
I = me.inertia(A, 0., 0., iZZ)
bodye = me.RigidBody('bodye', Dmc, A, m, (I, Dmc))
Poa = me.Particle('Poa', Po, mo)
BODY = [bodye, Poa]

FL = [(Dmc, -m*g*N.y), (Po, -mo*g*N.y)]

kd = [u - q.diff(t), umx - mx.diff(t), umy - my.diff(t)]
speed_constr = [umx - loesung[0], umy - loesung[1]]

q1 = [q, mx, my]
u_ind = [u]
u_dep = [umx, umy]

KM1 = me.KanesMethod(N, q_ind=q1, u_ind=u_ind, u_dependent=u_dep, kd_eqs=kd, 
        velocity_constraints=speed_constr)
(fr, frstar) = KM1.kanes_equations(BODY, FL)
MM1 = KM1.mass_matrix_full
force1 = KM1.forcing_full #.subs({sm.Derivative(x, t): rhsx})

force = sm.Matrix.vstack(force1, sm.Matrix([rhsx]))
force = force.subs({sm.Derivative(x, t): rhsx})
force = force.subs({sm.Derivative(x, t): rhsx})
print('force DS', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print('force has {} operations'.format(sum([force[i].count_ops(visual=False) 
                                             for i in range(len(force))])), '\n')

MM = sm.Matrix.hstack(MM1, sm.zeros(6, 1))
hilfs = sm.Matrix.hstack(sm.zeros(1, 6), sm.eye(1))
MM = sm.Matrix.vstack(MM, hilfs)
print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM has {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0]) 
        for j in range(MM.shape[1])])), '\n')

print(f'it took {time.time() - start1 :.3f} sec to establish Kanes equations')

Here the *sympy functions* are converted to *numpy functions* so numerical calculations may be done.\
Before, I calculate the functions for the *potential energy* and for the *kinetic energy*.

In [ ]:
pot_energie = m * g * me.dot(Dmc.pos_from(P0), N.y) + mo * g * me.dot(Po.pos_from(P0), N.y)
kin_energie = sum([koerper.kinetic_energy(N) for koerper in BODY])

qL = q1 + u_ind + u_dep + [x]
pL = [m, mo, g, a, b, iZZ, alpha, beta, amplitude, frequenz]

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

richtung_lam = sm.lambdify([mx, my] + [q, x] + pL, richtung, cse=True)
gesamt = gesamt.subs({CPx: x})
gesamt_lam = sm.lambdify([x] + pL, gesamt, cse=True)
loesung_lam = sm.lambdify([q, mx, my, x, u] + pL, loesung, cse=True)
loesung1_lam = sm.lambdify(qL + pL, loesung, cse=True)  # just for ease of plotting

pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)
kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)

r_max_lam = sm.lambdify([x] + pL, r_max, cse=True)

**Numerical integration**
- the parameters and the initial values of independent coordinates are set.
- the initial values of the dependent coordinates are calculated numerically.
- the ellipse must touch the street on exactly one point. I i check whether this condition is met, and an exception is raised if the minimal osculating circle of the street is smaller than the largest osculating circle of the ellipse.
- an exception is raised if $\alpha$ or $\beta$ are selected such that the particle will be outside of the ellipse. 

For stiff problems, method = *'Radau'* seems to be better than *no method*.


In [ ]:
#=============================================
# Input parameters
#=============================================
m1 = 1.
mo1 = 1.
g1 = 9.8
a1 = 1.
b1 = 2.
amplitude1 = 1.
frequenz1  = 0.275

alpha1 = 0.5
beta1 = 0.5

q11 = 0.5
u11 = 2.
x11 = -10.

intervall = 15.
#==============================================
start1 = time.time()

if alpha1**2/a1**2 + beta1**2/b1**2 >= 1.:
    raise Exception('Particle is outside the ellipse')

iZZ1 = 0.25 * m1 * (a1**2 + b1**2)   # from the internet
schritte = int(intervall * 30.)

pL_vals = [m1, mo1, g1, a1, b1, iZZ1, alpha1, beta1, amplitude1, frequenz1]

# numerically find the starting values of mx, my
def func1(x0, args):
    return richtung_lam(*x0, *args).reshape(2)
x0 = (x11, gesamt_lam(x11, *pL_vals) + b1)
args = [q11, x11] + pL_vals
for _ in range(3):
    mxmy, _, _, nachricht = fsolve(func1, x0, args, full_output=True, xtol=1.e-12)
    x0 = mxmy
mx1, my1 = mxmy
print(' {} To {:.4f} / {:.4f}. Accuracy is {} / {}'.format(nachricht, mx1, my1, func1(mxmy, args)[0], 
        func1(mxmy, args)[1]))

# get initial speed of Dmc
umx1, umy1 = loesung_lam(q11, mx1, my1, x11, u11, *pL_vals)
umx1, umy1 = umx1[0], umy1[0]
print('initial speed of Dmc: umx = {:.3f}, umy = {:.3f}'.format(umx1, umy1), '\n')

y0 = [q11, mx1, my1] + [u11] + [umx1, umy1] + [x11]
print('initial conditions are:')
print(y0, '\n')

#find the largest admissible r_max, given strasse, amplitude, frequenz
r_max = max(a1**2/b1, b1**2/a1)  # max osculating circle of an ellipse
def func2(x, args):
# just needed to get the arguments matching for minimize
    return np.abs(r_max_lam(x, *args))

x0 = 0.1            # initial guess
minimal = minimize(func2, x0, pL_vals)
if r_max < (x111 := minimal.get('fun')):
    print('selected r_max = {} is less than maximally admissible radius = {:.2f}, hence o.k.'
          .format(r_max, x111), '\n')
else:
    print('selected r_max {} is larger than admissible radius {:.2f}, hence NOT o.k.'
          .format(r_max, x111), '\n')
    raise Exception('the radius of the disc is too large')


def gradient(t, y, args):
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]


times = np.linspace(0., intervall, schritte)
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), method='Radau', atol=1.e-6,
    rtol=1.e-6)
resultat = resultat1.y.T
print(resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status])
print('the integration made {} function calls. It took {:.3f} sec'
      .format(resultat1.nfev, time.time() - start1))

Plot the generalized coordinates you want to see.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
bezeichnung = ['q', 'mx', 'my', 'u', 'umx', 'umy', 'x']
for i in (0, 1, 2):
    ax.plot(times, resultat[:, i], label=bezeichnung[i])
ax.set_xlabel('time (sec)')
ax.set_title('generalized coordinates')
ax.legend();

**Energies of the system**\
The total energy should be constant. I assume the small deviations are due to numerical errors, as making $a_{tol}, r_{tol}$ smaller reduces the deviation.

In [ ]:
kin_np = np.empty(schritte)
pot_np = np.empty(schritte)
total_np = np.empty(schritte)

for i in range(schritte):
    kin_np[i] = kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    pot_np[i] = pot_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    total_np[i] = kin_np[i] + pot_np[i]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, pot_np, label='potential energy')
ax.plot(times, kin_np, label='kinetic energy')
ax.plot(times, total_np, label='total energy')
ax.set_xlabel('time (sec)')
ax.set_ylabel("energy (Nm)")
ax.set_title('Energies of the system')
ax.legend();
total_max = np.max(total_np)
total_min = np.min(total_np)
print('max deviation of total energy from being constant is {:.4f} % of max total energy'.
      format((total_max - total_min)/total_max * 100))

*Violation of the speed constraints*\
Ideally, they should be zero.\
My formula may be a bit *circular* as I use the speeds calculated in the formula for the speed constraints. I do not know how else I should do it.

In [ ]:
deltaX_np = np.empty(schritte)
deltaY_np = np.empty(schritte)

for i in range(schritte):
    deltaX_np[i] = loesung1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
        *pL_vals)[0] - resultat[i, 4]
    deltaY_np[i] = loesung1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
        *pL_vals)[1] - resultat[i, 5]
    
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(times, deltaX_np, label='violation in X direction')
ax.plot(times, deltaY_np, label='violation in Y direction')
ax.set_xlabel('time (sec)')
ax.set_title('violation of speed constraints')
ax.legend();

Animate the motion of the ellipse

In [ ]:
#Animation

Dmcx = np.array([resultat[i, 1] for i in range(resultat.shape[0])])
Dmcy = np.array([resultat[i, 2] for i in range(resultat.shape[0])])

Po_lam = sm.lambdify(qL + pL, [me.dot(Po.pos_from(P0), uv) for uv in (N.x, N.y)])
Po_np = np.array([Po_lam(*[resultat[i, j] for j in range(7)], *pL_vals) for i in range(schritte)])

# needed to give the picture the right size.
xmin = np.min(Dmcx)
xmax = np.max(Dmcx)
ymin = np.min(Dmcy)
ymax = np.max(Dmcy)

# Data to draw the uneven street
cc = max(a1, b1)
strassex = np.linspace(xmin - 1.*cc, xmax + 1.*cc, schritte)
strassey = [gesamt_lam(strassex[i], *pL_vals) for i in range(schritte)]

test_np = np.sort(resultat[:, 6])
test1_np = np.array([gesamt_lam(test_np[i], *pL_vals) for i in range(schritte)])

if u11 > 0.:
    wohin = 'left'
else:
    wohin = 'right'

def animate_pendulum(times, x1, y1, z1):
    
    fig, ax = plt.subplots(figsize=(15, 15), subplot_kw={'aspect': 'equal'})
    
    ax.axis('on')
    ax.set_xlim(xmin - 1.*cc, xmax + 1.*cc)
    ax.set_ylim(ymin - 1.*cc, ymax + 1.*cc)
    ax.plot(strassex, strassey)
    
    ax.plot(test_np, test1_np, color='green')


    line1, = ax.plot([], [], 'o-', lw=0.5)                                      # center of the ellipse
    line2, = ax.plot([], [], 'o', color="black")                                # particle on the ellipse
    line3  = ax.axvline(resultat[0, 6], linestyle='--')                         # vertical tracking line
    line4  = ax.axhline(gesamt_lam(resultat[0, 6], *pL_vals), linestyle = '--') # horizontal trackimg line
    
    elli = patches.Ellipse((x1[0], y1[0]), width=2.*a1, height=2.*b1, angle=np.rad2deg(resultat[0
        , 0]), zorder=1, fill=True, color='red', ec='black')
    ax.add_patch(elli)

    def animate(i):
        message = (f'running time {times[i]:.2f} sec \n Initial speed is {np.abs(u11):.2f} radians/sec to the {wohin}'
            f'\n The dashed lines cross at the contact point \n The black dot is the particle')
        ax.set_title(message, fontsize=20)
        ax.set_xlabel('X direction', fontsize=20)
        ax.set_ylabel('Y direction', fontsize=20)
        elli.set_center((x1[i], y1[i]))
        elli.set_angle(np.rad2deg(resultat[i, 0]))
                       
        line1.set_data(x1[i], y1[i])                  
        line2.set_data(z1[i, 0], z1[i, 1])            
        line3.set_xdata([resultat[i, 6], resultat[i, 6]]) 
        wert = gesamt_lam(resultat[i, 6], *pL_vals)
        line4.set_ydata([wert, wert])
        return line1, line2, line3, line4,

    anim = animation.FuncAnimation(fig, animate, frames=schritte,
                                   interval=2000*np.max(times) / schritte,
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times, Dmcx, Dmcy, Po_np)
print(f'it took {time.time() - start :.3f} sec to run the program BEFORE HTML')
HTML(anim.to_jshtml())    # needed, when run on an iPad, I know no other way to do it. It is SLOW!